<a href="https://colab.research.google.com/github/noproblama25/inflation/blob/main/DataSourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Investigating inflation trends in the Netherlands

# 1. Inflation (target)

### 1.1 Data Sourcing

Source (ECB): https://data.ecb.europa.eu/data/datasets/ICP/ICP.M.BE.N.000000.4.ANR

In [ ]:
pip install eurostat

In [ ]:
import pandas as pd
import numpy as np
import eurostat
from datetime import datetime
import requests

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns

import json
from pandas import json_normalize
from io import StringIO

In [ ]:
COUNTRIES = ['NL', 'BE', 'DE']
START_YEAR = 1997
END_YEAR = 2025

def filter_dates(df, start_year=1997, end_year=2025):
    """Filter dataframe to date range"""
    if df.empty:
        return df

    # Find time column
    time_col = None
    for col in ['TIME_PERIOD', 'time', 'freq\\TIME_PERIOD']:
        if col in df.columns:
            time_col = col
            break

    if time_col is None:
        return df

    # Extract year from time column
    df['year'] = pd.to_datetime(df[time_col], errors='coerce').dt.year
    filtered = df[(df['year'] >= start_year) & (df['year'] <= end_year)].copy()
    filtered.drop('year', axis=1, inplace=True)

    return filtered

def get_all_eurostat_data():
    """Extract Eurostat data 1997-2025 for NL, BE, DE"""

    datasets = {}

    # CATEGORY 1: HICP Components
    try:
        df = eurostat.get_data_df('prc_hicp_midx',
                                  filter_pars={'geo': COUNTRIES,
                                              'coicop': ['CP00', 'NRG', 'FOOD', 'SERV', 'IGD_NNRG']})
        datasets['hicp_components'] = filter_dates(df, START_YEAR, END_YEAR)
    except:
        datasets['hicp_components'] = pd.DataFrame()

    # CATEGORY 2: Labor Market
    try:
        df = eurostat.get_data_df('une_rt_m',
                                  filter_pars={'geo': COUNTRIES, 's_adj': 'SA',
                                              'age': 'TOTAL', 'sex': 'T'})
        datasets['unemployment'] = filter_dates(df, START_YEAR, END_YEAR)
    except:
        datasets['unemployment'] = pd.DataFrame()

    try:
        df = eurostat.get_data_df('earn_mw_cur', filter_pars={'geo': COUNTRIES})
        datasets['minimum_wage'] = filter_dates(df, START_YEAR, END_YEAR)
    except:
        datasets['minimum_wage'] = pd.DataFrame()

    try:
        df = eurostat.get_data_df('lc_lci_r2_q',
                                  filter_pars={'geo': COUNTRIES, 's_adj': 'SCA',
                                              'nace_r2': 'B-S'})
        datasets['labor_costs'] = filter_dates(df, START_YEAR, END_YEAR)
    except:
        datasets['labor_costs'] = pd.DataFrame()

    try:
        df = eurostat.get_data_df('lc_ulc_r2_q',
                                  filter_pars={'geo': COUNTRIES, 's_adj': 'SCA',
                                              'nace_r2': 'B-S'})
        datasets['unit_labor_costs'] = filter_dates(df, START_YEAR, END_YEAR)
    except:
        datasets['unit_labor_costs'] = pd.DataFrame()

    # CATEGORY 4: Economic Growth
    try:
        df = eurostat.get_data_df('namq_10_gdp',
                                  filter_pars={'geo': COUNTRIES, 's_adj': 'SCA',
                                              'na_item': 'B1GQ', 'unit': 'CLV_PCH_PRE'})
        datasets['gdp_growth'] = filter_dates(df, START_YEAR, END_YEAR)
    except:
        datasets['gdp_growth'] = pd.DataFrame()

    try:
        df = eurostat.get_data_df('nama_10_lp_ulc',
                                  filter_pars={'geo': COUNTRIES, 'na_item': 'LPR_HW',
                                              'unit': 'PCH_PRE'})
        datasets['productivity'] = filter_dates(df, START_YEAR, END_YEAR)
    except:
        datasets['productivity'] = pd.DataFrame()

    return datasets

# Execute
data = get_all_eurostat_data()

# Access dataframes
hicp = data['hicp_components']
unemployment = data['unemployment']
min_wage = data['minimum_wage']
gdp = data['gdp_growth']


In [ ]:
gdp

In [ ]:
def get_ecb_inflation_data(countries=['NL', 'BE', 'DE'], start_year=1997):
    """Get HICP inflation from ECB Data Portal API"""

    components = {
        '000000': 'Total',
        'NRG': 'Energy',
        'FOOD': 'Food',
        'SERV': 'Services',
        'IGD_NNRG': 'Goods_excl_energy'
    }

    all_data = []

    for country in countries:
        for code, label in components.items():
            # Correct ECB SDMX API format
            url = f"https://data-api.ecb.europa.eu/service/data/ICP/M.{country}.N.{code}.4.ANR"

            params = {
                'startPeriod': f'{start_year}-01',
                'format': 'csvdata'
            }

            try:
                response = requests.get(url, params=params, timeout=30)
                if response.status_code == 200:
                    df = pd.read_csv(StringIO(response.text))
                    df['country'] = country
                    df['component'] = label
                    all_data.append(df)
            except:
                continue

    if not all_data:
        return pd.DataFrame()

    result = pd.concat(all_data, ignore_index=True)
    return result

# Execute
ecb_inflation = get_ecb_inflation_data(countries=['NL', 'BE', 'DE'], start_year=1997)



In [ ]:
# load VAT.csv and tobacco_excise_cleaned

vat = pd.read_csv('VAT.csv')
tobacco_excise = pd.read_csv('tobacco_excise_cleaned.csv')

### 1.2 Data Cleaning

In [ ]:
# check and display the headers of the various dataframes
# print dataframe name next to it

print(ecb_inflation.head())
print(hicp.head())
print(unemployment.head())
print(min_wage.head())
print(gdp.head())
print(vat.head())
print(tobacco_excise.head())


Cleanup of hicp dataframe

In [ ]:
# drop rows where coicop = 'IGD_NNRG' from dataframe
hicp = hicp[hicp['coicop'] != 'IGD_NNRG']

# Identify period columns
# Corrected column name: 'geo\TIME_PERIOD' contains the country codes
id_cols = ['coicop', 'geo\TIME_PERIOD', 'freq', 'unit'] # Add 'freq' and 'unit' to id_cols
period_cols = [col for col in hicp.columns if col not in id_cols]

df_hicp = hicp.melt(
    id_vars=['coicop', 'geo\TIME_PERIOD'], # Only these should be identifiers for melting
    value_vars=period_cols,
    var_name='PERIOD',
    value_name='VALUES'
)

# Rename the 'geo\TIME_PERIOD' column to 'geo' for easier access
df_hicp = df_hicp.rename(columns={'geo\TIME_PERIOD': 'geo'})

# Optionally convert PERIOD to datetime
df_hicp['PERIOD'] = pd.to_datetime(df_hicp['PERIOD'], format='%Y-%m')

# remove all records with PERIOD < 1997-01-01
df_hicp = df_hicp[df_hicp['PERIOD'] >= '1997-01-01']

df_hicp.head()

In [ ]:
print("Unique values in hicp['coicop']:")
display(df_hicp['coicop'].unique())

Cleanup of unemployment dataframe

In [ ]:
# convert date columns similar to earleir with hicp

# Identify identifier columns specific to the unemployment dataframe
# Based on `unemployment.head()` output, these are 'freq', 's_adj', 'age', 'unit', 'sex', 'geo\TIME_PERIOD'
unempl_id_cols = ['freq', 's_adj', 'age', 'unit', 'sex', 'geo\TIME_PERIOD']

# Identify period columns for unemployment dataframe
unempl_period_cols = [col for col in unemployment.columns if col not in unempl_id_cols]

df_unempl = unemployment.melt(
    id_vars=unempl_id_cols,
    value_vars=unempl_period_cols,
    var_name='PERIOD',
    value_name='VALUES'
)

# Rename the 'geo\TIME_PERIOD' column to 'geo' for easier access
df_unempl = df_unempl.rename(columns={'geo\TIME_PERIOD': 'geo'})

# Optionally convert PERIOD to datetime
df_unempl['PERIOD'] = pd.to_datetime(df_unempl['PERIOD'], format='%Y-%m', errors='coerce')

# Remove any rows where PERIOD conversion failed (if any) or VALUES are NaN
df_unempl = df_unempl.dropna(subset=['PERIOD', 'VALUES'])

# Remove any rows with date < 1997-01-01
df_unempl = df_unempl[df_unempl['PERIOD'] >= '1997-01-01']

# Remove any rows with UNIT <> PC_ACT
df_unempl = df_unempl[df_unempl['unit'] == 'PC_ACT']

# drop all columns except PERIOD, VALUES, geo
df_unempl = df_unempl[['PERIOD', 'VALUES', 'geo']]

df_unempl.head()

Cleanup of minimum wage

In [ ]:
min_wage

In [ ]:
# Step 1: Identify semi-annual columns (all columns with year pattern)
semi_annual_cols = [col for col in min_wage.columns if isinstance(col, str) and '-S' in col]

# Step 2: Create monthly dataframe
monthly_data = []

for idx, row in min_wage.iterrows():
    new_row = {
        'freq': row['freq'],
        'currency': row['currency'],
        'geo': row['geo\TIME_PERIOD']  # Note: using the actual column name
    }

    # Process each semi-annual period
    for col in semi_annual_cols:
        year = int(col.split('-')[0])
        semester = col.split('-')[1]

        # S1 = Jan-Jun, S2 = Jul-Dec
        months = range(1, 7) if semester == 'S1' else range(7, 13)

        # Fill all 6 months with semi-annual value
        for month in months:
            date_str = f"{year}-{month:02d}"
            new_row[date_str] = row[col]

    monthly_data.append(new_row)

df_minwage = pd.DataFrame(monthly_data)

# Step 3: Backfill 1997-1998 using 1999-01 value
for year in [1997, 1998]:
    for month in range(1, 13):
        date_str = f"{year}-{month:02d}"
        df_minwage[date_str] = df_minwage['1999-01']

# Step 4: Reorder columns chronologically
date_cols = sorted([col for col in df_minwage.columns if '-' in col and col not in ['freq', 'currency', 'geo']])
df_minwage = df_minwage[['freq', 'currency', 'geo'] + date_cols]

# only keep rows with currency = EUR
df_minwage = df_minwage[df_minwage['currency'] == 'EUR']

# Melt the df_minwage to long format
id_vars_minwage = ['freq', 'currency', 'geo']
value_vars_minwage = [col for col in df_minwage.columns if col not in id_vars_minwage]

df_minwage_melted = df_minwage.melt(
    id_vars=id_vars_minwage,
    value_vars=value_vars_minwage,
    var_name='PERIOD',
    value_name='VALUES'
)

# Convert 'PERIOD' to datetime
df_minwage_melted['PERIOD'] = pd.to_datetime(df_minwage_melted['PERIOD'], format='%Y-%m', errors='coerce')

# Remove rows with NaN values in 'VALUES' (e.g., for Germany before 2015)
df_minwage_melted.dropna(subset=['VALUES'], inplace=True)

# Filter to date range 1997-01-01 onwards if not already handled
df_minwage_melted = df_minwage_melted[df_minwage_melted['PERIOD'] >= '1997-01-01']

# Divide VALUES by 174 which is monthly working hours
df_minwage_melted['VALUES'] = df_minwage_melted['VALUES'] / 160

print(f"Shape of df_minwage_melted: {df_minwage_melted.shape}")
print(f"Date range: {df_minwage_melted['PERIOD'].min().strftime('%Y-%m')} to {df_minwage_melted['PERIOD'].max().strftime('%Y-%m')}")
print(f"\nFirst few rows:\n{df_minwage_melted.head()}")

Cleanup of GDP dataframe

In [ ]:
gdp

In [ ]:
# Identify identifier columns for the GDP dataframe
gdp_id_cols = ['freq', 'unit', 's_adj', 'na_item', 'geo\TIME_PERIOD']

# Identify period columns for GDP dataframe
gdp_period_cols = [col for col in gdp.columns if col not in gdp_id_cols]

df_gdp = gdp.melt(
    id_vars=gdp_id_cols,
    value_vars=gdp_period_cols,
    var_name='PERIOD',
    value_name='VALUES'
)

# Rename the 'geo\TIME_PERIOD' column to 'geo' for easier access
df_gdp = df_gdp.rename(columns={'geo\TIME_PERIOD': 'geo'})

# Convert 'PERIOD' to datetime objects. Eurostat quarterly data uses YYYY-QX format.
# We will convert it to the first day of the quarter.
df_gdp['PERIOD'] = df_gdp['PERIOD'].str.replace(r'(Q1)', '-01-01', regex=True)
df_gdp['PERIOD'] = df_gdp['PERIOD'].str.replace(r'(Q2)', '-04-01', regex=True)
df_gdp['PERIOD'] = df_gdp['PERIOD'].str.replace(r'(Q3)', '-07-01', regex=True)
df_gdp['PERIOD'] = df_gdp['PERIOD'].str.replace(r'(Q4)', '-10-01', regex=True)
df_gdp['PERIOD'] = pd.to_datetime(df_gdp['PERIOD'], errors='coerce')

# Remove any rows where PERIOD conversion failed (if any) or VALUES are NaN
df_gdp = df_gdp.dropna(subset=['PERIOD', 'VALUES'])

# Filter to date range 1997-01-01 onwards
df_gdp = df_gdp[df_gdp['PERIOD'] >= '1997-01-01']

print(f"Shape of df_gdp: {df_gdp.shape}")
print(f"Date range: {df_gdp['PERIOD'].min().strftime('%Y-%m')} to {df_gdp['PERIOD'].max().strftime('%Y-%m')}")
print(f"\nFirst few rows:\n{df_gdp.head()}")

In [ ]:
# drop all columns except freq, geo, period and values
df_gdp = df_gdp[['freq', 'geo', 'PERIOD', 'VALUES']]
df_gdp

Cleanup of VAT dataframe

In [ ]:
# transforming dataframe from yearly times series to monthly by filling the months with yearly value
vat_monthly = vat.copy()
vat_monthly = vat_monthly.rename(columns={'Jaar': 'year'}) # Rename 'Jaar' to 'year'
vat_monthly['month'] = 1 # Assume January for yearly data if no month is specified
vat_monthly['year'] = vat_monthly['year'].astype(str)
vat_monthly['month'] = vat_monthly['month'].astype(str)
vat_monthly['date'] = vat_monthly['year'] + '-' + vat_monthly['month'] + '-01'
vat_monthly['date'] = pd.to_datetime(vat_monthly['date'])
vat_monthly = vat_monthly.set_index('date')
vat_monthly = vat_monthly.resample('MS').ffill()
vat_monthly = vat_monthly.reset_index()


Accijnsen cleanup

In [ ]:
tobacco_excise

In [ ]:
# transforming dataframe from yearly times series to monthly by filling the months with yearly value
tobacco_excise_monthly = tobacco_excise.copy()
tobacco_excise_monthly = tobacco_excise_monthly.rename(columns={'Year': 'year'}) # Rename 'Year' to 'year'
tobacco_excise_monthly['month'] = 1 # Assume January for yearly data if no month is specified
tobacco_excise_monthly['year'] = tobacco_excise_monthly['year'].astype(str)
tobacco_excise_monthly['month'] = tobacco_excise_monthly['month'].astype(str)
tobacco_excise_monthly['date'] = tobacco_excise_monthly['year'] + '-' + tobacco_excise_monthly['month'] + '-01'
tobacco_excise_monthly['date'] = pd.to_datetime(tobacco_excise_monthly['date'])
tobacco_excise_monthly = tobacco_excise_monthly.set_index('date')
tobacco_excise_monthly = tobacco_excise_monthly.resample('MS').ffill()
tobacco_excise_monthly = tobacco_excise_monthly.reset_index()
tobacco_excise_monthly.head()

In [ ]:
# backfill 1997, 19998, 1999 creating values using data from 2000

# Get the tobacco excise values for the year 2000 (first month, as they are constant per year)
# Ensure 'year' column is int for proper filtering
tobacco_excise_monthly['year'] = tobacco_excise_monthly['year'].astype(int)
excise_2000 = tobacco_excise_monthly[tobacco_excise_monthly['year'] == 2000].iloc[0]

backfill_data = []
for year_to_backfill in [1997, 1998, 1999]:
    for month_num in range(1, 13): # For each month of the year
        date_str = f"{year_to_backfill}-{month_num:02d}-01"
        backfill_data.append({
            'date': pd.to_datetime(date_str),
            'year': year_to_backfill,
            'NL': excise_2000['NL'],
            'DE': excise_2000['DE'],
            'BE': excise_2000['BE'],
            'month': month_num
        })

df_backfill = pd.DataFrame(backfill_data)

# Concatenate the backfilled data with the original monthly data
tobacco_excise_monthly = pd.concat([df_backfill, tobacco_excise_monthly]).sort_values('date').reset_index(drop=True)

print(f"Shape of tobacco_excise_monthly after backfill: {tobacco_excise_monthly.shape}")
print(f"Date range: {tobacco_excise_monthly['date'].min().strftime('%Y-%m')} to {tobacco_excise_monthly['date'].max().strftime('%Y-%m')}")
print(f"\nFirst few rows:\n{tobacco_excise_monthly.head()}")

In [ ]:
tobacco_excise_monthly

# Graphs inflation figures

In [ ]:
# inflation per country

inflation_NL = ecb_inflation[ecb_inflation['country'] == 'NL']
inflation_BE = ecb_inflation[ecb_inflation['country'] == 'BE']
inflation_DE = ecb_inflation[ecb_inflation['country'] == 'DE']

# transpose column OBS_VALUE to the serie per TIME_PERIOD

NL_serie = inflation_NL.pivot(index='TIME_PERIOD', columns='component', values='OBS_VALUE')
BE_serie = inflation_BE.pivot(index='TIME_PERIOD', columns='component', values='OBS_VALUE')
DE_serie = inflation_DE.pivot(index='TIME_PERIOD', columns='component', values='OBS_VALUE')

# combine countries series into a single dataframe

inflation_all = pd.concat([NL_serie, BE_serie, DE_serie], axis=1)


In [ ]:
# plot of inflation

# Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare the data for plotting
# Filter ecb_inflation for the 'Total' component
df_plot = ecb_inflation[ecb_inflation['component'] == 'Total'].copy()

# Convert 'TIME_PERIOD' to datetime objects for proper plotting
df_plot['TIME_PERIOD'] = pd.to_datetime(df_plot['TIME_PERIOD'])

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_plot, x='TIME_PERIOD', y='OBS_VALUE', hue='country')
plt.title('Inflation Trends Over Time by Country (Total HICP)')
plt.xlabel('Date')
plt.ylabel('Inflation (%)')
plt.grid(False)

# Set the x-axis major locator and formatter
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator(1)) # Show a tick every year
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y')) # Format the date as YYYY

# Set the x-axis limits to match the data range
ax.set_xlim(df_plot['TIME_PERIOD'].min(), df_plot['TIME_PERIOD'].max())

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# plot series of component of dataframe df_hicp as a line
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns

# Define a mapping for coicop codes to more descriptive labels
coicop_label_map = {
    'CP00': 'Total HICP',
    'NRG': 'Energy',
    'FOOD': 'Food',
    'SERV': 'Services'
    # 'IGD_NNRG' was dropped earlier in data cleaning, so it's not included here
}

# Create a new column with descriptive coicop labels for plotting
df_hicp_plot = df_hicp.copy()
df_hicp_plot['coicop_label'] = df_hicp_plot['coicop'].map(coicop_label_map)

# Create the FacetGrid
g = sns.FacetGrid(
    df_hicp_plot,
    col='coicop_label',
    col_wrap=2,
    height=4,
    aspect=1.2,
    sharey=False,
    hue='geo' # Pass hue to FacetGrid to define color mapping for all subplots
)

# Map the line plot to each facet
g.map_dataframe(sns.lineplot, x='PERIOD', y='VALUES')

# Set overall title for the plot
g.fig.suptitle('Inflation Trends by Component and Country', y=1.02, fontsize=16)

# Improve labels for each subplot
for ax in g.axes.flat:
    ax.set_xlabel('Date')
    ax.set_ylabel('Index Value')
    ax.tick_params(axis='x', rotation=45) # Rotate x-axis labels

    # Set the x-axis major locator and formatter for each subplot
    ax.xaxis.set_major_locator(mdates.YearLocator(5)) # Show a tick every 5 years
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y')) # Format the date as YYYY

    # Add a legend to each subplot
    # This should now work correctly because map_dataframe with hue sets labels on artists
    ax.legend(title='Country', loc='best')

g.set_titles(col_template="{col_name}") # Use the descriptive labels as facet titles
g.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout to make space for the suptitle
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Create the line plot for minimum wage
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_minwage_melted, x='PERIOD', y='VALUES', hue='geo')

plt.title('Minimum Wage Trends Over Time by Country (EUR/hour)')
plt.xlabel('Date')
plt.ylabel('Minimum Wage (EUR/hour)')
plt.grid(True)

# Set the x-axis major locator and formatter
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator(5)) # Show a tick every 5 years
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y')) # Format the date as YYYY

# Set the x-axis limits to match the data range
ax.set_xlim(df_minwage_melted['PERIOD'].min(), df_minwage_melted['PERIOD'].max())

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:


# summarize VALUES per year
df_gdp_summary = df_gdp.groupby(['geo', pd.Grouper(key='PERIOD', freq='YE')])['VALUES'].sum().reset_index()

# Extract just the year for plotting for cleaner x-axis labels
df_gdp_summary['YEAR'] = df_gdp_summary['PERIOD'].dt.year

plt.figure(figsize=(15, 8))
ax = sns.barplot(data=df_gdp_summary, x='YEAR', y='VALUES', hue='geo', palette='Set1') # Changed palette to 'Set1'

plt.title('Annual GDP Growth by Country', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Annual GDP Growth (%)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add numerical values to the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f')

plt.tight_layout()
plt.show()